In [ ]:
!pip install numpy pandas scikit-learn dowhy causal-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.1/403.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.0/193.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.6.7
    Uninstalling cvxpy-1.6.7:
      Successfully uninstalled cvxpy-1.6.7


In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)
n = 2000

Z = np.random.normal(size=n)              # confounder
X1 = Z + np.random.normal(size=n)
X2 = np.random.normal(size=n)
Y  = 2*X1 + 3*X2 + 1.5*Z + np.random.normal(size=n)
X3 = Y + np.random.normal(size=n)          # anti-causal

df = pd.DataFrame({
    "Z": Z,
    "X1": X1,
    "X2": X2,
    "X3": X3,
    "Y": Y
})


In [ ]:
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.cit import fisherz

data = df[["Z", "X1", "X2", "X3", "Y"]].values
cg = pc(data, alpha=0.05, indep_test=fisherz)

print(cg.G)


  0%|          | 0/5 [00:00<?, ?it/s]

Graph Nodes:
X1;X2;X3;X4;X5

Graph Edges:
1. X1 --- X2
2. X1 --> X5
3. X2 --> X5
4. X3 --> X5
5. X5 --> X4



In [ ]:
from dowhy import CausalModel

selected_features = []

for feature in ["Z", "X1", "X2", "X3"]:
    model = CausalModel(
        data=df,
        treatment=feature,
        outcome="Y",
        common_causes=[c for c in ["Z", "X1", "X2", "X3"] if c != feature]
    )

    estimand = model.identify_effect()
    estimate = model.estimate_effect(
        estimand,
        method_name="backdoor.linear_regression"
    )

    if abs(estimate.value) > 0.2:
        selected_features.append(feature)

print("Causally selected features:", selected_features)


Causally selected features: ['Z', 'X1', 'X2', 'X3']


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def intervene(df, feature, value):
    df_int = df.copy()
    df_int[feature] = value
    return df_int

baseline_model = LinearRegression().fit(
    df[selected_features], df["Y"]
)

baseline_preds = baseline_model.predict(df[selected_features])

for f in selected_features:
    df_int = intervene(df, f, df[f].mean() + 2)
    preds = baseline_model.predict(df_int[selected_features])
    print(f, "Δ prediction mean:", np.mean(preds - baseline_preds))


Z Δ prediction mean: 1.4289094040218533
X1 Δ prediction mean: 1.9179038037705127
X2 Δ prediction mean: 2.918396120014052
X3 Δ prediction mean: 1.0451477018161208


In [ ]:
final_model = LinearRegression()
final_model.fit(df[selected_features], df["Y"])

print("Coefficients:")
for f, c in zip(selected_features, final_model.coef_):
    print(f, round(c, 3))


Coefficients:
Z 0.714
X1 0.959
X2 1.459
X3 0.523
